# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [6]:
orders = pd.read_csv(r"C:\Users\kim.buchner\Desktop\Ironhack\Module1\WEEK2\DAY3\lab-df-calculation-and-transformation\data\Orders.csv",index_col = 0)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [8]:
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [11]:
#Add amount spent per customer
customers = orders.groupby(["CustomerID"])[["amount_spent"]].sum()

In [15]:
customers["Customer Types"] = pd.qcut(customers["amount_spent"], q=[0, 0.75, 0.95, 1], labels=["Cheap Customers","Preferred Customers","VIP Customers"])

In [16]:
customers.groupby()

,amount_spent,Customer Types
CustomerID,,
12346,77183.60,VIP Customers
12347,4310.00,Preferred Customers
12348,1797.24,Preferred Customers
12349,1757.55,Preferred Customers
12350,334.40,Cheap Customers
...,...,...
18280,180.60,Cheap Customers
18281,80.82,Cheap Customers
18282,178.05,Cheap Customers


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [28]:
customers_country = orders.groupby(["CustomerID","Country"])[["amount_spent"]].sum()

customers_country["Customer Types"] = pd.qcut(customers_country["amount_spent"], q=[0, 0.75, 0.95, 1], labels=["Cheap Customers","Preferred Customers","VIP Customers"])

customers_country = customers_country.reset_index()

customers_country.loc[customers_country['Customer Types'] == 'VIP Customers']['Country'].value_counts()

United Kingdom     178
Germany             11
France               9
Switzerland          3
EIRE                 2
Spain                2
Portugal             2
Japan                2
Finland              1
Australia            1
Cyprus               1
Singapore            1
Netherlands          1
Channel Islands      1
Denmark              1
Sweden               1
Norway               1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [31]:
customers_country.loc[(customers_country['Customer Types'] == 'VIP Customers') | (customers_country['Customer Types'] == 'Preferred Customers')]['Country'].value_counts()

United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Norway               7
Spain                7
Portugal             7
Finland              5
Italy                5
Australia            4
Japan                4
Channel Islands      4
Denmark              3
EIRE                 3
Cyprus               3
Sweden               2
Israel               2
Netherlands          1
Iceland              1
Austria              1
Lebanon              1
Poland               1
Singapore            1
Canada               1
Greece               1
Malta                1
Name: Country, dtype: int64